In [1]:
from __future__ import print_function
from vowpalwabbit import pyvw

In [2]:
tag = {}
tag['CC'] = 1
tag['CD'] = 2
tag['DT'] = 3
tag['EX'] = 3
tag['FW'] = 4
tag['IN'] = 5
tag['JJ'] = 6
tag['JJR'] = 6
tag['JJS'] = 6
tag['MD'] = 7
tag['NN'] = 8
tag['NNP'] = 8
tag['NNPS'] = 8
tag['NNS'] = 8
tag['PDT'] = 3
tag['POS'] = 11
tag['PRP'] = 9
tag['PRP$'] = 9
tag['RB'] = 10
tag['RBR'] = 10
tag['RBS'] = 10
tag['RP'] = 11
tag['SYM'] = 12
tag['TO'] = 11
tag['UH'] = 13
tag['VB'] = 14
tag['VBD'] = 14
tag['VBG'] = 14
tag['VBN'] = 14
tag['VBP'] = 14
tag['VBZ'] = 14
tag['WDT'] = 3
tag['WP'] = 9
tag['WP$'] = 9
tag['WRB'] = 10

In [3]:
s = ''
punc = ['#', '$',"''",'(',')',',','.',':',"``"]
sentence = []
sentences = []
pos_tags = set([])
file = open('train.txt', 'r') 
for line in file:
    if line == '\n': 
        sentences.append(sentence)
        sentence = []
        continue
    parts = line.split()
    if parts[1] in punc:
        continue
    pos_tags.add(parts[1])
    tup = (tag[parts[1]],parts[0])
    sentence.append(tup)

In [13]:
sentence_test = []
sentences_test = []
pos_tags_test = set([])
file = open('test.txt', 'r') 
for line in file:
    if line == '\n': 
        sentences_test.append(sentence_test)
        sentence_test = []
        continue
    parts = line.split()
    if parts[1] in punc:
        continue
    pos_tags_test.add(parts[1])
    tup = (tag[parts[1]],parts[0])
    sentence_test.append(tup)

In [ ]:
sentences[0][1][0]

In [ ]:
class SequenceLabeler(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        # you must must must initialize the parent class
        # this will automatically store self.sch <- sch, self.vw <- vw
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        
        # set whatever options you want
        sch.set_options( sch.AUTO_HAMMING_LOSS | sch.AUTO_CONDITION_FEATURES )

    def _run(self, sentence):   # it's called _run to remind you that you shouldn't call it directly!
        output = []
        for n in range(len(sentence)):
            pos,word = sentence[n]
            # use "with...as..." to guarantee that the example is finished properly
            with self.vw.example({'w': [word]}) as ex:
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'), (n-1, 'q')])
                output.append(pred)
        return output

## Customizing features


In [40]:
class SequenceLabeler2(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        sch.set_options(sch.AUTO_HAMMING_LOSS)

    def _run(self, sentence):
        output = []
        #loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            prevPred = output[n-1] if n > 0 else '<s>'
            prev_n_2 = output[n-2] if n > 1 else '<s>'
            posnew, prevWord = sentence[n-1] if n> 0 else ('<s>','<s>')
            with self.vw.example({'w': [word],'p': [prevPred], 'r':[prevWord]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n,'r')])
                output.append(pred)
                
        return output
    
#sequenceLabeler2 = vw.init_search_task(SequenceLabeler2)
#sequenceLabeler2.learn(my_dataset)
#print(sequenceLabeler2.predict( [(0,w) for w in "the sandwich ate a monster".split()] ))

## Customizing loss functions

In [47]:
wt = {}

wt[1] = 1.0 
wt[2] = 1.0
wt[3] = 1.0
wt[4] = 1.0
wt[5] = 1.0
wt[6] = 1.1
wt[7] = 1.0
wt[8] = 1.0
wt[9] = 1.0
wt[10] = 1.1
wt[11] = 1.0
wt[12] = 1.0
wt[13] = 1.0
wt[14] = 1.1

In [125]:
class SequenceLabeler3(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        sch.set_options(sch.AUTO_CONDITION_FEATURES)

    def _run(self, sentence):
        output = []
        loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            with self.vw.example({'w': [word]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n-1,'q')])
                output.append(pred)
                if pred != pos:
                    loss += wt_err[pos]            
        self.sch.loss(loss)
                
        return output

In [126]:
def train_loss():
    print("Calculating training loss...")
    loss_t=[]
    s=0
    for sen in sentences[:1000]:
        s+=1
        temp=[]
        for w in sen:
            temp.append((1,w[1]))
        out = sequenceLabeler3.predict(temp)
        #print(out)
        loss=0
        c=0
        for i in out:        
            if sen[c][0] in tag_count:
                tag_count[sen[c][0]] += 1
            else:
                tag_count[sen[c][0]] = float(1)
                tag_error[sen[c][0]] = float(0)

            if i != sen[c][0]:
                loss+=1
                tag_error[sen[c][0]] += 1
                #print ("Predicted value at ",c," is ",i)
                #print (sentences[0][c])
            c+=1
        loss_t.append(loss/float(c))

    print ("Training Loss:")
    print (sum(loss_t) / float(len(loss_t)))

In [127]:
def test_loss():
    print("Calculating loss...")
    loss_t=[]
    s=0
    for sen in sentences_test:
        s+=1
        if(s%1000==0):
            print(s)
        temp=[]
        for w in sen:
            temp.append((1,w[1]))
        out = sequenceLabeler3.predict(temp)
        #print(out)
        loss=0
        c=0
        for i in out:        
            if i != sen[c][0]:
                loss+=1
                #print ("Predicted value at ",c," is ",i)
                #print (sentences[0][c])
            c+=1
        loss_t.append(loss/float(c))

    print ("Test loss")
    print (sum(loss_t) / float(len(loss_t)))

In [128]:
import operator
def updateWeights():
    print("Updating weights...")
    tag_loss = {}
    sum_err=0
    for t in tag_count:
        tag_loss[t] = tag_error[t]/tag_count[t]
        sum_err+=tag_error[t]
        
    sorted_loss = sorted(tag_loss.items(), key=operator.itemgetter(1))

    for tp in sorted_loss:
            print( tp[0], " = ", tag_error[tp[0]], "/", tag_count[tp[0]], " = ", tp[1] )
            wt_err[tp[0]]=tp[1]*(tag_error[tp[0]]/sum_err)
            
    avg_err=sum(wt_err)/14.0
    wt_err[:]=[1+(x - avg_err) for x in wt_err]
    
        
        
        

In [129]:
vw = pyvw.vw("--search 14 --audit --quiet --search_task hook --ring_size 1024 -f pos_tagger.model")

In [130]:
sequenceLabeler3 = vw.init_search_task(SequenceLabeler3)

In [131]:
wt_err=[1]*15
tag_count = {}
tag_error = {}

In [132]:
for i in range(4):
    print (i)
    sequenceLabeler3.learn(sentences[:1000])
    tag_count = {}
    tag_error = {}
    train_loss()
    test_loss()
    updateWeights()

0
Calculating training loss...
Training Loss:
0.0479161243961
Calculating loss...
1000
2000
Test loss
0.136344372717
Updating weights...
1  =  0.0 / 575.0  =  0.0
7  =  1.0 / 235.0  =  0.00425531914894
5  =  27.0 / 2587.0  =  0.0104367993815
11  =  9.0 / 806.0  =  0.0111662531017
9  =  12.0 / 585.0  =  0.0205128205128
3  =  67.0 / 2190.0  =  0.0305936073059
8  =  260.0 / 7383.0  =  0.0352160368414
14  =  224.0 / 3170.0  =  0.0706624605678
6  =  176.0 / 1715.0  =  0.102623906706
2  =  106.0 / 871.0  =  0.121699196326
10  =  98.0 / 783.0  =  0.125159642401
4  =  4.0 / 7.0  =  0.571428571429
1
Calculating training loss...
Training Loss:
0.0449351382448
Calculating loss...
1000
2000
Test loss
0.136486973228
Updating weights...
1  =  0.0 / 575.0  =  0.0
4  =  0.0 / 7.0  =  0.0
7  =  0.0 / 235.0  =  0.0
5  =  14.0 / 2587.0  =  0.00541167375338
11  =  6.0 / 806.0  =  0.00744416873449
9  =  8.0 / 585.0  =  0.0136752136752
3  =  78.0 / 2190.0  =  0.0356164383562
8  =  274.0 / 7383.0  =  0.03711

In [ ]:
class SequenceLabeler4(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        sch.set_options(sch.AUTO_CONDITION_FEATURES)

    def _run(self, sentence):
        output = []
        loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            with self.vw.example({'w': [word]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n-1,'q')])
                output.append(pred)
                if pred != pos:
                    loss += wt_err[pos]            
        self.sch.loss(loss)
                
        return output

In [ ]:
out = sequenceLabeler2.predict(temp)
print(out)
loss=0
c=0
for i in out:
    if i != sentences[0][c][0]:
        loss+=1
        print ("Predicted value at ",c," is ",i)
        print (sentences[0][c])
    c+=1



In [ ]:
sentences_test[0]